# Preparing Price Data

## Imports & Settings

In [1]:
import warnings
warnings.filterwarnings('ignore')

from pathlib import Path
import requests
from io import BytesIO
from zipfile import ZipFile, BadZipFile
import time
import itertools
from collections import Counter
from datetime import datetime, timedelta
from tqdm import tqdm
import json

import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml

import yfinance as yf

import datasets

In [2]:
sec_path = Path('sec-edgar-10k')

data_store = sec_path / 'assets.h5'
sec_parsed_data_path = sec_path / 'parsed-data'
companies_data_path = sec_path / 'subset-companies'

DAYS_BEFORE = 93
DAYS_AFTER = 31

## Quandl Wiki Prices

> Quandl has been [acuqired by NASDAQ](https://www.nasdaq.com/about/press-center/nasdaq-acquires-quandl-advance-use-alternative-data) in late 2018. In 2021, NASDAQ [integrated Quandl's data platform](https://data.nasdaq.com/). Free US equity data is still available under a [new URL](https://data.nasdaq.com/databases/WIKIP/documentation), subject to the limitations mentioned below.

[NASDAQ](https://data.nasdaq.com/) makes available a [dataset](/home/stefan/drive/machine-learning-for-trading/data/create_datasets.ipynb) with stock prices, dividends and splits for 3,000 US publicly-traded companies. Prior to its acquisition (April 11, 2018), Quandl announced the end of community support (updates). The historical data are useful as a first step towards demonstrating the application of the machine learning solutions, just ensure you design and test your own algorithms using current, professional-grade data.

1. Follow the instructions to create a free [NASDAQ account](https://data.nasdaq.com/sign-up)
2. [Download](https://data.nasdaq.com/tables/WIKIP/WIKI-PRICES/export) the entire WIKI/PRICES data
3. Extract the .zip file
4. Move to this directory and rename to wiki_prices.csv
5. Run the below code to store in fast HDF format

In [3]:
df = pd.read_csv(sec_path / 'wiki_prices.csv', parse_dates=['date'], index_col=['date', 'ticker'], infer_datetime_format=True).sort_index()
df.info()

with pd.HDFStore(data_store) as store:
    store.put('quandl/wiki/prices', df)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 15389314 entries, (Timestamp('1962-01-02 00:00:00'), 'ARNC') to (Timestamp('2018-03-27 00:00:00'), 'ZUMZ')
Data columns (total 12 columns):
 #   Column       Dtype  
---  ------       -----  
 0   open         float64
 1   high         float64
 2   low          float64
 3   close        float64
 4   volume       float64
 5   ex-dividend  float64
 6   split_ratio  float64
 7   adj_open     float64
 8   adj_high     float64
 9   adj_low      float64
 10  adj_close    float64
 11  adj_volume   float64
dtypes: float64(12)
memory usage: 1.4+ GB
None


### Wiki Prices Metadata

Quandl used to make some stock meta data be available on its website. Instead of using the Quandl API, load the file `wiki_stocks.csv` as described and store in HDF5 format.

In [4]:
df = pd.read_csv(sec_path / 'wiki_stocks.csv')
df.info()

with pd.HDFStore(data_store) as store:
    store.put('quandl/wiki/stocks', df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3199 entries, 0 to 3198
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   code    3199 non-null   object
 1   name    3199 non-null   object
dtypes: object(2)
memory usage: 50.1+ KB
None


## Metadata on US Companies

In [5]:
df = pd.read_csv(sec_path / 'us_equities_meta_data.csv')
df.info()

with pd.HDFStore(data_store) as store:
    store.put('us_equities/stocks', df.set_index('ticker'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6834 entries, 0 to 6833
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ticker     6834 non-null   object 
 1   name       6834 non-null   object 
 2   lastsale   6718 non-null   float64
 3   marketcap  5766 non-null   float64
 4   ipoyear    3038 non-null   float64
 5   sector     5288 non-null   object 
 6   industry   5288 non-null   object 
dtypes: float64(3), object(4)
memory usage: 373.9+ KB


## Download and Persist Stock Prices and SEC Filings Info

Here we only download price data for those companies whose `cik` exist in the SEC 10-K filing dataset for the available filing years only.

In [6]:
filing_index = pd.read_csv(sec_path / 'filing_index.csv', parse_dates=['DATE_FILED']).rename(columns=str.lower)
filing_index.index += 1

In [7]:
cik_year = set([tuple(f[1].values) for f in filing_index[['cik', 'year']].iterrows()])

In [8]:
dataset = datasets.load_from_disk(sec_parsed_data_path)
train_parsed = dataset['train']

In [9]:
filtered = train_parsed.filter(lambda example: (int(example['cik']), int(example['year'])) in cik_year)

In [10]:
filing_filtered = filing_index.loc[filing_index.cik.astype(str).isin(filtered['cik']), :]

In [11]:
filing_filtered.head()

,cik,company_name,form_type,date_filed,edgar_link,quarter,ticker,sic,exchange,hits,year
1,1000180,SANDISK CORP,10-K,2013-02-19,edgar/data/1000180/0001000180-13-000009.txt,1,SNDK,3572,NASDAQ,3,2013
2,1000209,MEDALLION FINANCIAL CORP,10-K,2013-03-13,edgar/data/1000209/0001193125-13-103504.txt,1,TAXI,6199,NASDAQ,0,2013
3,1000228,HENRY SCHEIN INC,10-K,2013-02-13,edgar/data/1000228/0001000228-13-000010.txt,1,HSIC,5047,NASDAQ,3,2013
4,1000229,CORE LABORATORIES N V,10-K,2013-02-19,edgar/data/1000229/0001000229-13-000009.txt,1,CLB,1389,NYSE,2,2013
5,1000232,KENTUCKY BANCSHARES INC KY,10-K,2013-03-28,edgar/data/1000232/0001104659-13-025094.txt,1,KTYB,6022,OTC,0,2013


### Download stock price data using Yfinance

In [12]:
def yfinance_download(symbol, cik, filing_dates, days_before, days_after):
    prices = []
    ticker = yf.Ticker(symbol)
    
    for filing, date in filing_dates.to_dict().items():
        start = date - timedelta(days=days_before)
        end = date + timedelta(days=days_after)
        
        try:
            df = ticker.history(start=start, end=end)
        except:
            print("Yfinance lost connection...")
            return []
            
        prices.append(df.assign(ticker=symbol, cik=cik, filing=filing))

    return prices

In [13]:
print(f"Storing stock price data for {len(filing_filtered.ticker.unique())} tickers between years {filing_filtered.year.min()}-{filing_filtered.year.max()}:")

Storing stock price data for 6219 tickers between years 2013-2016:


In [24]:
yf_data, missing, lost = [], [], []

for i, (symbol, dates) in tqdm(enumerate(filing_filtered.groupby('ticker').date_filed, 1)):
    cik = filing_filtered.loc[filing_filtered.ticker == symbol, 'cik'].iloc[0]
    
    price_hist = yfinance_download(symbol, cik, dates, days_before=DAYS_BEFORE, days_after=DAYS_AFTER)

    if not price_hist:
        lost.append(symbol)
    elif price_hist[0].empty:
        missing.append(symbol)
    else:
        yf_data.append(price_hist)

    time.sleep(0.5)

2it [00:02,  1.06s/it]AAC: Data doesn't exist for startDate = 1372478400, endDate = 1383192000
AAC: Data doesn't exist for startDate = 1403928000, endDate = 1414641600
AAC: Data doesn't exist for startDate = 1436673600, endDate = 1447390800
6it [00:04,  1.27it/s]AAMC: Data doesn't exist for startDate = 1352178000, endDate = 1362891600
AAMC: Data doesn't exist for startDate = 1384837200, endDate = 1395547200
AAMC: Data doesn't exist for startDate = 1417237200, endDate = 1427947200
AAMC: Data doesn't exist for startDate = 1448686800, endDate = 1459396800
8it [00:06,  1.27it/s]AAN: Data doesn't exist for startDate = 1353474000, endDate = 1364184000
AAN: Data doesn't exist for startDate = 1385182800, endDate = 1395892800
AAN: Data doesn't exist for startDate = 1417237200, endDate = 1427947200
AAN: Data doesn't exist for startDate = 1448686800, endDate = 1459396800
12it [00:09,  1.22it/s]AAPH: No timezone found, symbol may be delisted
AAPH: No timezone found, symbol may be delisted
14it [00

In [26]:
yf_data = itertools.chain.from_iterable(yf_data)

In [27]:
yf_data = pd.concat(yf_data).rename(columns=str.lower)

In [28]:
yf_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 884210 entries, 2013-09-17 00:00:00-04:00 to 2016-04-13 00:00:00-04:00
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   open           884207 non-null  float64
 1   high           884207 non-null  float64
 2   low            884207 non-null  float64
 3   close          884207 non-null  float64
 4   volume         884210 non-null  float64
 5   dividends      884210 non-null  float64
 6   stock splits   884210 non-null  float64
 7   ticker         884210 non-null  object 
 8   cik            884210 non-null  int64  
 9   filing         884210 non-null  int64  
 10  capital gains  53119 non-null   float64
 11  adj close      0 non-null       float64
dtypes: float64(9), int64(2), object(1)
memory usage: 87.7+ MB


In [29]:
yf_data.drop(columns=['dividends', 'stock splits', 'capital gains', 'adj close'], inplace=True)

In [30]:
yf_data.to_hdf(sec_path / 'sec_returns.h5', 'data/yfinance')

In [33]:
print(f"{len(missing)} ticker price data are missing. Try to retrieve from Quandl stock prices:")

3422 ticker price data are missing. Try to retrieve from Quandl stock prices:


### Get missing prices from Quandl if exist

In [34]:
to_do = filing_filtered.loc[~filing_filtered.ticker.isin(yf_data.ticker.unique()), ['ticker', 'date_filed']]

In [35]:
min_year = to_do.date_filed.min().year

In [38]:
quandl_tickers = pd.read_hdf(data_store, 'quandl/wiki/prices').loc[pd.IndexSlice[f'{min_year}':, :], :].index.unique('ticker')
quandl_tickers = list(set(quandl_tickers).intersection(set(to_do.ticker)))

In [40]:
print(f"Found {len(quandl_tickers)} ticker price data in Quandl stock prices.") 

Found 1230 ticker price data in Quandl stock prices.


In [41]:
to_do = filing_filtered.loc[filing_filtered.ticker.isin(quandl_tickers), ['ticker', 'date_filed']]

In [43]:
ohlcv = ['adj_open', 'adj_high', 'adj_low', 'adj_close', 'adj_volume']

In [44]:
quandl = pd.read_hdf(data_store, 'quandl/wiki/prices').loc[pd.IndexSlice[f'{min_year}': , quandl_tickers], ohlcv].rename(columns=lambda x: x.replace('adj_', ''))

In [75]:
quandl_data = []

for i, (symbol, dates) in tqdm(enumerate(to_do.groupby('ticker').date_filed, 1)):
    cik = filing_filtered.loc[filing_filtered.ticker == symbol, 'cik'].iloc[0]
        
    for filing, date in dates.to_dict().items():
        start = date - timedelta(days=DAYS_BEFORE)
        end = date + timedelta(days=DAYS_AFTER)
        try:
            quandl_data.append(quandl.loc[pd.IndexSlice[start:end, symbol], :].reset_index('ticker').assign(cik=cik, filing=filing))
        except:
            continue
        
quandl_data = pd.concat(quandl_data)

1230it [00:09, 129.86it/s]


In [76]:
quandl_data.to_hdf(sec_path / 'sec_returns.h5', 'data/quandl')

### Combine, clean and persist

In [95]:
data = pd.concat(
    [
        pd.read_hdf(sec_path / 'sec_returns.h5', 'data/yfinance'),
        pd.read_hdf(sec_path / 'sec_returns.h5', 'data/quandl')
    ],
    axis=0
)

In [96]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1228824 entries, 2013-09-17 00:00:00-04:00 to 2015-01-23 00:00:00
Data columns (total 8 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   open    1228802 non-null  float64
 1   high    1228820 non-null  float64
 2   low     1228820 non-null  float64
 3   close   1228821 non-null  float64
 4   volume  1228824 non-null  float64
 5   ticker  1228824 non-null  object 
 6   cik     1228824 non-null  int64  
 7   filing  1228824 non-null  int64  
dtypes: float64(5), int64(2), object(1)
memory usage: 84.4+ MB


In [97]:
data = data.loc[:, ['filing', 'ticker', 'cik', 'open', 'high', 'low', 'close', 'volume']]

In [98]:
data.to_hdf(sec_path / 'sec_returns.h5', 'prices')

### Create a subset of companies for out-of-sample return predictions [Optional]

In [82]:
SUBSET_TICKERS = ["AAPL", "MSFT", "AMZN", "WMT", "XOM", "PZZA", "VIPS", "AX", "UHS", "SMCI", "HEES", "KAR", "TNC", "SD", "VNET"]

In [83]:
json_file = open(sec_path / 'company_tickers.json', 'r')
tickers = json.load(json_file)
json_file.close()

tickers = pd.DataFrame(tickers).T
tickers = tickers.loc[tickers.ticker.isin(SUBSET_TICKERS)].reset_index(drop=True)
tickers['cik_url'] = tickers.cik_str.apply(
    lambda cik: [f"https://data.sec.gov/submissions/CIK{cik:010d}.json",
                 f"https://data.sec.gov/submissions/CIK{cik:010d}-submissions-001.json"]
)
tickers['cik'] = tickers.cik_str.astype(int)
tickers.cik_str = tickers.cik_str.apply(lambda cik: f"CIK{cik:010d}")

In [84]:
df_10k = pd.DataFrame()

for cik in tickers.cik_str.to_list():
    cik_int = tickers.loc[tickers.cik_str==cik].cik.values[0]
    ticker = tickers.loc[tickers.cik_str==cik].ticker.values[0]
    try:
        json_file = open((companies_data_path / f'{cik}.json').as_posix(), 'r')
        values = json.load(json_file)
        json_file.close()
        df = pd.DataFrame(values['filings']['recent'])
    except Exception as e:
        print(f"File {ticker}:{cik}.json does not exist.")
        continue

    try:
        json_file = open((companies_data_path / f'{cik}-submissions-001.json').as_posix(), 'r')
        values = json.load(json_file)
        json_file.close()
        df = pd.concat([df, pd.DataFrame(values)], axis=0)
    except Exception as e:
        print(f"File {ticker}:{cik}-submissions-001.json does not exist.")

    df = df[df.form == '10-K']['filingDate']
    df.reset_index(drop=True, inplace=True)
    df = pd.concat(
        [df,
         pd.Series([cik_int] * df.shape[0], name='cik'),
         pd.Series([ticker] * df.shape[0], name='ticker'),
        ],
        axis=1
    )
    df_10k = pd.concat([df_10k, df], axis=0)
df_10k.filingDate = pd.to_datetime(df_10k.filingDate)

File SMCI:CIK0001375365-submissions-001.json does not exist.
File VIPS:CIK0001529192-submissions-001.json does not exist.
File VNET:CIK0001508475-submissions-001.json does not exist.


In [85]:
df_10k.groupby('cik').min().set_index('ticker').join(df_10k.groupby('cik').max().set_index('ticker'), lsuffix='_start', rsuffix='_end')

,filingDate_start,filingDate_end
ticker,,
XOM,2004-03-15,2023-02-22
TNC,2002-03-29,2023-02-23
WMT,2006-03-29,2023-03-17
AAPL,1994-12-13,2022-10-28
UHS,1994-03-30,2023-02-27
MSFT,2006-08-25,2023-07-27
PZZA,1997-03-31,2023-02-23
AMZN,1999-03-05,2023-02-03
AX,2005-09-16,2022-09-08


In [86]:
df_10k = df_10k.loc[(df_10k.filingDate.dt.year >= 2010) & (df_10k.filingDate.dt.year <= 2020)]

In [87]:
yf_data, missing, lost = [], [], []

for i, (symbol, dates) in tqdm(enumerate(df_10k.groupby('ticker').filingDate, 1)):
    cik = df_10k.loc[df_10k.ticker == symbol, 'cik'].iloc[0]
    
    price_hist = yfinance_download(symbol, cik, dates, days_before=DAYS_BEFORE, days_after=DAYS_AFTER)

    if not price_hist:
        lost.append(symbol)
    elif price_hist[0].empty:
        missing.append(symbol)
    else:
        yf_data.append(price_hist)

    time.sleep(0.5)

7it [00:10,  1.48s/it]SD: Data doesn't exist for startDate = 1451278800, endDate = 1461988800
SD: Data doesn't exist for startDate = 1416978000, endDate = 1427688000
SD: Data doesn't exist for startDate = 1385528400, endDate = 1396238400
SD: Data doesn't exist for startDate = 1354078800, endDate = 1364788800
SD: Data doesn't exist for startDate = 1322283600, endDate = 1332993600
SD: Data doesn't exist for startDate = 1290834000, endDate = 1301544000
SD: Data doesn't exist for startDate = 1259384400, endDate = 1270094400
13it [00:20,  1.59s/it]


In [88]:
yf_data = itertools.chain.from_iterable(yf_data)

In [89]:
yf_data = pd.concat(yf_data).rename(columns=str.lower)

In [90]:
yf_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 11455 entries, 2020-07-29 00:00:00-04:00 to 2010-03-26 00:00:00-04:00
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   open          11455 non-null  float64
 1   high          11455 non-null  float64
 2   low           11455 non-null  float64
 3   close         11455 non-null  float64
 4   volume        11455 non-null  float64
 5   dividends     11455 non-null  float64
 6   stock splits  11455 non-null  float64
 7   ticker        11455 non-null  object 
 8   cik           11455 non-null  int64  
 9   filing        11455 non-null  int64  
 10  adj close     0 non-null      float64
dtypes: float64(8), int64(2), object(1)
memory usage: 1.0+ MB


In [93]:
yf_data.drop(columns=['dividends', 'stock splits', 'adj close'], inplace=True)

In [94]:
yf_data.to_hdf(sec_path / 'sec_returns.h5', 'data/subset/yfinance')